In [367]:
from pynq import Overlay
from pynq import MMIO
from time import sleep
ol = Overlay("/home/xilinx/jupyter_notebooks/puf/PUF_new_v1_wrapper.bit")
ol.download()

puf_ip=MMIO(0x43C00000, 0x10000)

puf_ip.write(0x10, 10) # challenge 1
puf_ip.write(0x18, 6) # challlenge 2
puf_ip.write(0x28, 0b00000) # tuner 1st part (32 bits)
#puf_ip.write(0x2c, 0xFF) # tuner 2nd part (8 bits)
print("starting")
puf_ip.write(0x44, 0x01) # start bit 
puf_ip.write(0x00, 1)
print("input 1: ", puf_ip.read(0x10)) 
print("input 2: ", puf_ip.read(0x18)) 
print("product 1: ", puf_ip.read(0x30)) 
print("product 2: ", puf_ip.read(0x38))
print("result: ", puf_ip.read(0x20)) # this is reading response (arbiter)
print("result in binary: ", "{0:08b}".format(puf_ip.read(0x20)))

starting
input 1:  10
input 2:  6
product 1:  60
product 2:  60
result:  20
result in binary:  00010100


In [61]:
x=open("values.txt","w+")

for k in range(0,32):
    for i in range(0,16):
        for j in range(0,16):

            puf_ip.write(0x10, i) # challenge 1
            puf_ip.write(0x18, j) # challlenge 2
            puf_ip.write(0x28, k) # tuner 1st part (32 bits)
            #puf_ip.write(0x2c, 0xFF) # tuner 2nd part (8 bits)
            #print("starting")
            puf_ip.write(0x44, 0x01) # start bit 
            puf_ip.write(0x00, 1)
            x.write("input 1: ")
            x.write(str(puf_ip.read(0x10))) 
            x.write(" input 2: ")
            x.write(str(puf_ip.read(0x18))) 
            #print("product 1: ", puf_ip.read(0x30)) 
            x.write(" product : ") 
            x.write(str(puf_ip.read(0x38)))
            x.write(" result: " ) 
            x.write(str(puf_ip.read(0x20))) # this is reading response (arbiter) 
            x.write(" tune values : ") 
            x.write(str(k))
            x.write('\n')
            puf_ip.write(0x44,0x00)
            
x.close()

In [62]:
x=open("values_2643.txt","w+")

for k in range(0,32):

    puf_ip.write(0x10, 6) # challenge 1
    puf_ip.write(0x18, 2) # challlenge 2
    puf_ip.write(0x28, k) # tuner 1st part (32 bits)
            #puf_ip.write(0x2c, 0xFF) # tuner 2nd part (8 bits)
            #print("starting")
    puf_ip.write(0x44, 0x01) # start bit 
    puf_ip.write(0x00, 1)
    x.write("input 1: ")
    x.write(str(puf_ip.read(0x10))) 
    x.write(" input 2: ")
    x.write(str(puf_ip.read(0x18))) 
            #print("product 1: ", puf_ip.read(0x30)) 
    x.write(" product : ") 
    x.write(str(puf_ip.read(0x38)))
    x.write(" result: " ) 
    x.write(str(puf_ip.read(0x20))) # this is reading response (arbiter) 
    x.write(" tune values : ") 
    x.write(str(k))
    x.write('\n')
    puf_ip.write(0x44,0x00)
    
    puf_ip.write(0x10, 4) # challenge 1
    puf_ip.write(0x18, 3) # challlenge 2
    puf_ip.write(0x28, k) # tuner 1st part (32 bits)
            #puf_ip.write(0x2c, 0xFF) # tuner 2nd part (8 bits)
            #print("starting")
    puf_ip.write(0x44, 0x01) # start bit 
    puf_ip.write(0x00, 1)
    x.write("input 1: ")
    x.write(str(puf_ip.read(0x10))) 
    x.write(" input 2: ")
    x.write(str(puf_ip.read(0x18))) 
            #print("product 1: ", puf_ip.read(0x30)) 
    x.write(" product : ") 
    x.write(str(puf_ip.read(0x38)))
    x.write(" result: " ) 
    x.write(str(puf_ip.read(0x20))) # this is reading response (arbiter) 
    x.write(" tune values : ") 
    x.write(str(k))
    x.write('\n')
    
    puf_ip.write(0x44,0x00)
x.close()

In [215]:
import statistics
l=np.zeros((32,8))
for k in range(0,32):
    for i in range(0,16):
        for j in range(0,16):
            puf_ip.write(0x10, i) # challenge 1
            puf_ip.write(0x18, j) # challlenge 2
            puf_ip.write(0x28, k) # tuner 1st part (32 bits)
            
            puf_ip.write(0x44, 0x01) # start bit 
            puf_ip.write(0x00, 1)
            a="{0:08b}".format(puf_ip.read(0x20))
            puf_ip.write(0x44,0x00)
            for m in range(0,8):
                l[k][m]=l[k][m]+int(a[m])

In [366]:
for i in range(0,32):
    print(l[i][1])

IndexError: string index out of range

In [129]:
v=0
for i in range(0,256):
    a="{0:08b}".format(l[i])
    v=v+int(a[0])

ValueError: Unknown format code 'b' for object of type 'float'

In [120]:
v

128

In [394]:

# generate .xdc file

x = open("constraints2.xdc", "w+")
l=['A','B','C','D']
number=35
for i in range(8):
    for j in range(16):
        if j%4==0:
            number=number+1
        path = "PUF_final_i/PUF_0/inst/PUF_id"+"["+str(i)+"].top/Tune/Coarse"+"["+str(j)+"].PC/Coarse"
        x.write("set_property BEL "+ l[j%4]+"LUT6 [get_cells \"" + path +"\"]")
        x.write('\n')
        x.write("set_property LOC SLICE_X"+str(number)+"Y90 "+ "[get_cells \""+ path + "\"]")
        x.write('\n')
        x.write('\n')


number=35
for i in range(0,8):
    for j in range(16):
        if j%4==0:
            number=number+1
        path = "PUF_final_i/PUF_0/inst/PUF_id"+"["+str(i)+"].bottom/Tune/Coarse"+"["+str(j)+"].PC/Coarse"
        x.write("set_property BEL "+ l[j%4]+"LUT6 [get_cells \"" + path +"\"]")
        x.write('\n')
        x.write("set_property LOC SLICE_X"+str(number)+"Y92 "+ "[get_cells \""+ path + "\"]")
        x.write('\n')
        x.write('\n')



number=36
for i in range(8):
    path = "PUF_final_i/PUF_0/inst/PUF_id"+"["+str(i)+"].arbiter"
    #x.write("set_property BEL "+ l[j%4]+"LUT6 [get_cells \"" + path +"\"]")
    #x.write('\n')
    x.write("set_property LOC SLICE_X"+ str(number)+"Y1 "+ "[get_cells \""+ path + "\"]")
    x.write('\n')
    x.write('\n')
    number=number+1

x.close()


-- ------------------------Address Info-------------------
-- 0x00 : Control signals
--        bit 0  - ap_start (Read/Write/SC)
--        bit 1  - ap_done (Read/COR)
--        bit 2  - ap_idle (Read)
--        bit 3  - ap_ready (Read)
--        bit 7  - auto_restart (Read/Write)
--        others - reserved
-- 0x04 : Global Interrupt Enable Register
--        bit 0  - Global Interrupt Enable (Read/Write)
--        others - reserved
-- 0x08 : IP Interrupt Enable Register (Read/Write)
--        bit 0  - Channel 0 (ap_done)
--        others - reserved
-- 0x0c : IP Interrupt Status Register (Read/TOW)
--        bit 0  - Channel 0 (ap_done)
--        others - reserved
-- 0x10 : Data signal of challenge1axi_V
--        bit 3~0 - challenge1axi_V[3:0] (Read/Write)
--        others  - reserved
-- 0x14 : reserved
-- 0x18 : Data signal of challenge2axi_V
--        bit 3~0 - challenge2axi_V[3:0] (Read/Write)
--        others  - reserved
-- 0x1c : reserved
-- 0x20 : Data signal of responseaxi_V
--        bit 7~0 - responseaxi_V[7:0] (Read)
--        others  - reserved
-- 0x24 : Control signal of responseaxi_V
--        bit 0  - responseaxi_V_ap_vld (Read/COR)
--        others - reserved
-- 0x28 : Data signal of tuneraxi_top_1_V
--        bit 31~0 - tuneraxi_top_1_V[31:0] (Read/Write)
-- 0x2c : reserved
-- 0x30 : Data signal of tuneraxi_top_2_V
--        bit 31~0 - tuneraxi_top_2_V[31:0] (Read/Write)
-- 0x34 : reserved
-- 0x38 : Data signal of tuneraxi_top_3_V
--        bit 31~0 - tuneraxi_top_3_V[31:0] (Read/Write)
-- 0x3c : reserved
-- 0x40 : Data signal of tuneraxi_top_4_V
--        bit 31~0 - tuneraxi_top_4_V[31:0] (Read/Write)
-- 0x44 : reserved
-- 0x48 : Data signal of tuneraxi_btm_1_V
--        bit 31~0 - tuneraxi_btm_1_V[31:0] (Read/Write)
-- 0x4c : reserved
-- 0x50 : Data signal of tuneraxi_btm_2_V
--        bit 31~0 - tuneraxi_btm_2_V[31:0] (Read/Write)
-- 0x54 : reserved
-- 0x58 : Data signal of tuneraxi_btm_3_V
--        bit 31~0 - tuneraxi_btm_3_V[31:0] (Read/Write)
-- 0x5c : reserved
-- 0x60 : Data signal of tuneraxi_btm_4_V
--        bit 31~0 - tuneraxi_btm_4_V[31:0] (Read/Write)
-- 0x64 : reserved
-- 0x68 : Data signal of output1axi_V
--        bit 7~0 - output1axi_V[7:0] (Read)
--        others  - reserved
-- 0x6c : Control signal of output1axi_V
--        bit 0  - output1axi_V_ap_vld (Read/COR)
--        others - reserved
-- 0x70 : Data signal of output2axi_V
--        bit 7~0 - output2axi_V[7:0] (Read)
--        others  - reserved
-- 0x74 : Control signal of output2axi_V
--        bit 0  - output2axi_V_ap_vld (Read/COR)
--        others - reserved
-- 0x78 : Data signal of startaxi_V
--        bit 0  - startaxi_V[0] (Read/Write)
--        others - reserved
-- 0x7c : 

In [74]:
from pynq import Overlay
from pynq import MMIO
from time import sleep
ol = Overlay("/home/xilinx/jupyter_notebooks/puf/long_puf_wrapper.bit")
ol.download()

puf_ip=MMIO(0x43C00000, 0x10000)
challenge1 = 0x10
challenge2 = 0x18
response = 0x20
tuner_top_1 = 0x28
tuner_top_2 = 0x30
tuner_top_3 = 0x38
tuner_top_4 = 0x40
tuner_btm_1 = 0x48
tuner_btm_2 = 0x50
tuner_btm_3 = 0x58
tuner_btm_4 = 0x60
output1 = 0x68
output2 = 0x70
startaxi = 0x78

puf_ip.write(challenge1, 11) # challenge 1
puf_ip.write(challenge2, 9) # challlenge 2
puf_ip.write(tuner_top_1, 0xFFFFFFFF) 
puf_ip.write(tuner_top_2, 0) 
puf_ip.write(tuner_top_3, 0) 
puf_ip.write(tuner_top_4, 0) 
puf_ip.write(tuner_btm_1, 0xFFFFFFFF) 
puf_ip.write(tuner_btm_2, 0xFFFFFFFF) 
puf_ip.write(tuner_btm_3, 0xFFFFFFFF) 
puf_ip.write(tuner_btm_4, 0xFFFFFFFF) 
puf_ip.write(0x00, 1)
print("starting")
puf_ip.write(startaxi, 0b1) # start bit 
puf_ip.write(0x00, 1)

print("input 1: ", puf_ip.read(challenge1)) 
print("input 2: ", puf_ip.read(challenge2)) 
print("product 1: ", puf_ip.read(output1)) 
print("product 2: ", puf_ip.read(output2))
print("result: ", puf_ip.read(response)) # this is reading response (arbiter) 

starting
input 1:  11
input 2:  9
product 1:  99
product 2:  99
result:  33


In [257]:
from pynq import Overlay
from pynq import MMIO
from time import sleep
ol = Overlay("/home/xilinx/jupyter_notebooks/puf/PUF_try2_wrapper.bit")
ol.download()
puf_ip=MMIO(0x43C00000, 0x10000)

x=open("values1.txt","w+")

for k in range(0,32):
    for i in range(0,16):
        for j in range(0,16):

            puf_ip.write(0x10, i) # challenge 1
            puf_ip.write(0x18, j) # challlenge 2
            puf_ip.write(0x28, k) # tuner 1st part (32 bits)
            #puf_ip.write(0x2c, 0xFF) # tuner 2nd part (8 bits)
            #print("starting")
            puf_ip.write(0x40, 0x01) # start bit 
            puf_ip.write(0x00, 1)
            x.write("input 1: ")
            x.write(str(puf_ip.read(0x10))) 
            x.write(" input 2: ")
            x.write(str(puf_ip.read(0x18))) 
            #print("product 1: ", puf_ip.read(0x30)) 
            x.write(" product : ") 
            x.write(str(puf_ip.read(0x38)))
            x.write(" result: " ) 
            x.write(str(puf_ip.read(0x20))) # this is reading response (arbiter) 
            x.write(" tune values : ") 
            x.write(str(k))
            x.write('\n')
            puf_ip.write(0x40,0x00)
            
x.close()


In [258]:
import statistics
import numpy as np
l=np.zeros((32,8))
v=0
avg=[]
for k in range(0,32):
    v=0
    for i in range(0,16):
        for j in range(0,16):
            puf_ip.write(0x10, i) # challenge 1
            puf_ip.write(0x18, j) # challlenge 2
            puf_ip.write(0x28, k) # tuner 1st part (32 bits)
            
            puf_ip.write(0x44, 0x01) # start bit 
            puf_ip.write(0x00, 1)
            a="{0:08b}".format(puf_ip.read(0x20))
            puf_ip.write(0x44,0x00)
            for m in range(0,8):
                l[k][m]=l[k][m]+int(a[m])

In [279]:
v=np.ndarray(256)
s=0
for i in range(0,16):
    for j in range(0,16):
        puf_ip.write(0x10, i) # challenge 1
        puf_ip.write(0x18, j) # challlenge 2
        puf_ip.write(0x28, 1) # tuner 1st part (32 bits)
            
        puf_ip.write(0x44, 0x01) # start bit 
        puf_ip.write(0x00, 1)
        a="{0:08b}".format(puf_ip.read(0x20))
        puf_ip.write(0x44,0x00)
        v[s]=a
        s=s+1

In [341]:
v1=np.ndarray(256)
s=0
from pynq import Overlay
from pynq import MMIO
from time import sleep
ol = Overlay("/home/xilinx/jupyter_notebooks/puf/PUF_try2_wrapper.bit")
ol.download()
puf_ip=MMIO(0x43C00000, 0x10000)

for i in range(0,16):
    for j in range(0,16):

        puf_ip.write(0x10, i) # challenge 1
        puf_ip.write(0x18, j) # challlenge 2
        puf_ip.write(0x28, 1) # tuner 1st part (32 bits)
            
        puf_ip.write(0x44, 0x01) # start bit 
        puf_ip.write(0x00, 1)
        v1[s]=int("{0:08b}".format(puf_ip.read(0x20)))
        puf_ip.write(0x44,0x00)
        s=s+1

In [365]:
x=open("vals.txt","w+")
for i in range(0,256):
    x.write(str(v1[i]))
    x.write("\n")
    
x.close()

In [340]:
bin(115),bin(100),bin(23)

('0b1110011', '0b1100100', '0b10111')

 0x00 : Control signals
//        bit 0  - ap_start (Read/Write/SC)
//        bit 1  - ap_done (Read/COR)
//        bit 2  - ap_idle (Read)
//        bit 3  - ap_ready (Read)
//        bit 7  - auto_restart (Read/Write)
//        others - reserved
// 0x04 : Global Interrupt Enable Register
//        bit 0  - Global Interrupt Enable (Read/Write)
//        others - reserved
// 0x08 : IP Interrupt Enable Register (Read/Write)
//        bit 0  - Channel 0 (ap_done)
//        others - reserved
// 0x0c : IP Interrupt Status Register (Read/TOW)
//        bit 0  - Channel 0 (ap_done)
//        others - reserved
// 0x10 : Data signal of challenge1axi_V
//        bit 3~0 - challenge1axi_V[3:0] (Read/Write)
//        others  - reserved
// 0x14 : reserved
// 0x18 : Data signal of challenge2axi_V
//        bit 3~0 - challenge2axi_V[3:0] (Read/Write)
//        others  - reserved
// 0x1c : reserved
// 0x20 : Data signal of responseaxi_V
//        bit 7~0 - responseaxi_V[7:0] (Read)
//        others  - reserved
// 0x24 : Control signal of responseaxi_V
//        bit 0  - responseaxi_V_ap_vld (Read/COR)
//        others - reserved
// 0x28 : Data signal of tuneraxi_V
//        bit 4~0 - tuneraxi_V[4:0] (Read/Write)
//        others  - reserved
// 0x2c : reserved
// 0x30 : Data signal of output1axi_V
//        bit 7~0 - output1axi_V[7:0] (Read)
//        others  - reserved
// 0x34 : Control signal of output1axi_V
//        bit 0  - output1axi_V_ap_vld (Read/COR)
//        others - reserved
// 0x38 : Data signal of output2axi_V
//        bit 7~0 - output2axi_V[7:0] (Read)
//        others  - reserved
// 0x3c : Control signal of output2axi_V
//        bit 0  - output2axi_V_ap_vld (Read/COR)
//        others - reserved
// 0x40 : Data signal of startaxi_V
//        bit 0  - startaxi_V[0] (Read/Write)
//        others - reserved
// 0x44 : reserved
// (SC = Self Clear, COR = Clear on Read, TOW = Toggle on Write, COH = Clear on Handshake)


In [389]:
from pynq import Overlay
from pynq import MMIO
from time import sleep
ol = Overlay("/home/xilinx/jupyter_notebooks/puf/PUF_final_wrapper.bit")
ol.download()
puf_ip=MMIO(0x43C00000, 0x10000)
import statistics
import numpy as np
l2=np.zeros((32,8))
v=0
avg=[]
for k in range(0,32):
    v=0
    for i in range(0,16):
        for j in range(0,16):
            puf_ip.write(0x10, i) # challenge 1
            puf_ip.write(0x18, j) # challlenge 2
            puf_ip.write(0x28, k) # tuner 1st part (32 bits)
            
            puf_ip.write(0x40, 0x01) # start bit 
            puf_ip.write(0x00, 1)
            a="{0:08b}".format(puf_ip.read(0x20))
            puf_ip.write(0x40,0x00)
            for m in range(0,8):
                l2[k][m]=l2[k][m]+int(a[m])

In [392]:
for i in range(0,32):
    print(l2[i][0])

112.0
128.0
128.0
128.0
128.0
128.0
128.0
128.0
128.0
128.0
128.0
128.0
128.0
128.0
128.0
128.0
128.0
128.0
128.0
128.0
128.0
128.0
128.0
128.0
128.0
128.0
128.0
128.0
128.0
128.0
128.0
128.0
